# Setup

In [1]:
import os
import numpy as np
import rasterio
import geopandas as gpd
from rasterio import features
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

In [3]:
def load_raster(path):
    """Load raster and return array + profile."""
    src = rasterio.open(path)
    arr = src.read(1)
    profile = src.profile
    return arr, profile

def save_raster(path, array, profile):
    """Save raster with given profile."""
    profile.update(dtype=rasterio.uint8, count=1)
    with rasterio.open(path, "w", **profile) as dst:
        dst.write(array.astype("uint8"), 1)

def rasterize_shapefile(gdf, reference_profile, attribute):
    """Rasterize shapefile according to reference raster."""
    shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf[attribute])]
    raster = features.rasterize(
        shapes=shapes,
        out_shape=(reference_profile["height"], reference_profile["width"]),
        transform=reference_profile["transform"],
        fill=0,
        dtype="uint8"
    )
    return raster

In [5]:
PATCH_SIZE = 128  # can change to 64 or 256
BATCH_SIZE = 8
EPOCHS = 30

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Number of bands for the model (edit later)
NUM_FEATURE_BANDS = 8

Using device: cpu


# Load data

In [ ]:
BASE_DIR = r"data/12-training-data/input"


In [ ]:
DEPENDENT_DIR = os.path.join(BASE_DIR, "dependent-data-urban-no")
LANDCOVER_DIR = os.path.join(BASE_DIR, "final-data-land-cover")

# Feature rasters live directly inside BASE_DIR
FEATURE_DIR = BASE_DIR

OUTPUT_DIR = 

In [8]:
rasters_2019 = {
    "NDVI": os.path.join(FEATURE_DIR, "NDVI_2019.tif"),
    "NDBI": os.path.join(FEATURE_DIR, "NDBI_2019.tif"),
    "MNDWI": os.path.join(FEATURE_DIR, "MNDWI_2019.tif"),
    "BSI": os.path.join(FEATURE_DIR, "BSI_2019.tif"),
    "DEM": os.path.join(FEATURE_DIR, "DEM-DAegu-merged-clipmask.tif"),
    "slope": os.path.join(FEATURE_DIR, "slope.tif"),
    "hillshade": os.path.join(FEATURE_DIR, "hillshade.tif"),
    "night": os.path.join(FEATURE_DIR, "VNL_2019_daegu_clipped.tif"),
    "ecology": os.path.join(FEATURE_DIR, "env-assessment-data-2019.tif"),
}

In [9]:
feature_arrays = {}
feature_profiles = {}

print("Loading 2019 rasters...\n")

for name, path in rasters_2019.items():
    arr, profile = load_raster(path)
    feature_arrays[name] = arr
    feature_profiles[name] = profile
    print(f"{name} loaded → shape: {arr.shape}")


Loading 2019 rasters...

NDVI loaded → shape: (4531, 3714)
NDBI loaded → shape: (4531, 3714)
MNDWI loaded → shape: (4531, 3714)
BSI loaded → shape: (4531, 3714)
DEM loaded → shape: (1622, 1330)
slope loaded → shape: (1622, 1330)
hillshade loaded → shape: (1622, 1330)
night loaded → shape: (99, 100)
ecology loaded → shape: (4590, 3684)


In [10]:
urb19_path = os.path.join(DEPENDENT_DIR, "landcover_2019_urban_binary.shp")
urb24_path = os.path.join(DEPENDENT_DIR, "landcover_2024_urban_binary.shp")

gdf_urb19 = gpd.read_file(urb19_path)
gdf_urb24 = gpd.read_file(urb24_path)

print("Urban 2019 polygons:", len(gdf_urb19))
print("Urban 2024 polygons:", len(gdf_urb24))



Loading dependent variable shapefiles...
Urban 2019 polygons: 332700
Urban 2024 polygons: 923467


In [11]:
lc19_path = os.path.join(LANDCOVER_DIR, "landcover_2019_final.shp")
lc24_path = os.path.join(LANDCOVER_DIR, "landcover_2024_final.shp")

gdf_lc19 = gpd.read_file(lc19_path)
gdf_lc24 = gpd.read_file(lc24_path)

print("Landcover 2019 classes:", len(gdf_lc19))
print("Landcover 2024 classes:", len(gdf_lc24))

Landcover 2019 classes: 332700
Landcover 2024 classes: 923467


In [12]:
reference_profile = feature_profiles["NDVI"]
print("\nReference raster info:")
print("  Size:", reference_profile["height"], "x", reference_profile["width"])
print("  CRS:", reference_profile["crs"])
print("  Resolution:", reference_profile["transform"][0])



Reference raster info:
  Size: 4531 x 3714
  CRS: EPSG:32652
  Resolution: 10.0


# Rasterize urban shapfile

In [15]:
urban_2019_raster_path = os.path.join(FEATURE_DIR, "urban_2019.tif")
urban_2024_raster_path = os.path.join(FEATURE_DIR, "urban_2024.tif")